In [6]:
import pandas as pd
import sqlite3


In [7]:
#importing data stored in the database to pandas dataframe

database_name = 'RecommendationDatabase.db'
conn = sqlite3.connect(database_name)

# Define SQL query to select all data from the table
sql_query_ratings = "SELECT * FROM Ratings;"
sql_query_movies = "SELECT * FROM Movies"

# Read data from SQLite database into a pandas DataFrame
reviews = pd.read_sql_query(sql_query_ratings, conn)
movies = pd.read_sql_query(sql_query_movies, conn)


conn.close()

In [8]:
reviews.head()

,user_id,movie_id,ratings,Time Stamp,Date
0,1,1074638,7,1365029107,2013-04-03 18:45:07
1,1,1853728,8,1366576639,2013-04-21 16:37:19
2,2,0104257,8,1364690142,2013-03-30 20:35:42
3,2,1259521,8,1364118447,2013-03-24 05:47:27
4,2,1991245,7,1364117717,2013-03-24 05:35:17


In [9]:
movies.head()

,movie_id,movie,genre,Year,1800's,1900's,2000's,Horror,Sci-Fi,Fantasy,...,Documentary,Sport,Crime,Drama,Western,Musical,War,News,Adult,Romance
0,0002844,Fantômas - À l'ombre de la guillotine (1913),Crime|Drama,1913,0,1,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
1,0004936,The Bank (1915),Comedy|Short,1915,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0007264,The Rink (1916),Comedy|Short,1916,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0008133,The Immigrant (1917),Short|Comedy|Drama|Romance,1917,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
4,0009968,Broken Blossoms or The Yellow Man and the Girl...,Drama|Romance,1919,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1


In [12]:
#making a new dataframe that has ranked movies according to mean rating, number of times it was rated, and the last time it was rated.

def create_ranked_df(movies, reviews):
    
        movie_ratings = reviews.groupby('movie_id')['ratings']
        avg_ratings = movie_ratings.mean()
        num_ratings = movie_ratings.count()
        last_rating = pd.DataFrame(reviews.groupby('movie_id').max()['Date'])
        last_rating.columns = ['last_rating']

        # Add Dates
        rating_count_df = pd.DataFrame({'avg_rating': avg_ratings, 'num_ratings': num_ratings})
        rating_count_df = rating_count_df.join(last_rating)

        # merge with the movies dataset
        movie_recs = movies.set_index('movie_id').join(rating_count_df)

        # sort by top avg rating and number of ratings
        ranked_movies = movie_recs.sort_values(['avg_rating', 'num_ratings', 'last_rating'], ascending=False)

        # for edge cases - subset the movie list to those with only 5 or more reviews
        ranked_movies = ranked_movies[ranked_movies['num_ratings'] > 4]
        
        return ranked_movies

ranked_movies = create_ranked_df(movies, reviews)

In [13]:
ranked_movies.head()

,movie,genre,Year,1800's,1900's,2000's,Horror,Sci-Fi,Fantasy,Music,...,Drama,Western,Musical,War,News,Adult,Romance,avg_rating,num_ratings,last_rating
movie_id,,,,,,,,,,,,,,,,,,,,,
2737018,Selam (2013),Drama,2013,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,10.000000,7,2013-05-06 04:09:39
1869425,Filly Brown (2012),Drama|Music,2012,0,0,1,0,0,0,1,...,1,0,0,0,0,0,0,10.000000,5,2013-04-26 11:32:55
2592910,CM101MMXI Fundamentals (2013),Comedy,2013,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,9.833333,6,2013-03-25 17:22:18
0056172,Lawrence of Arabia (1962),Adventure|Biography|Drama|History|War,1962,0,1,0,0,0,0,0,...,1,0,0,1,0,0,0,9.666667,9,2013-05-19 15:23:40
0080339,Airplane! (1980),Comedy|Romance,1980,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,9.600000,5,2013-05-25 23:14:38


In [14]:
#making top n recommendation of any/new user

def popular_recommendations(user_id, n_top, ranked_movies):
    
    top_movies = list(ranked_movies['movie'][:n_top])

    return top_movies

In [19]:
recs_20_for_43 = popular_recommendations('43', 20, ranked_movies) #top 20 movies recommendation for the user with id 43.
print(recs_20_for_1)

['Selam (2013)', 'Filly Brown (2012)', 'CM101MMXI Fundamentals (2013)', 'Lawrence of Arabia (1962)', 'Airplane! (1980)', 'Yi dai zong shi (2013)', 'True Grit (1969)', 'The Princess Bride (1987)', 'Home Alone (1990)', 'The Shawshank Redemption (1994)', 'The Godfather (1972)', 'Metropolis (1927)', 'A Bridge Too Far (1977)', 'Il buono, il brutto, il cattivo. (1966)', 'My Name Is Khan (2010)', 'Double Indemnity (1944)', 'Akira (1988)', 'Terminator 2: Judgment Day (1991)', "Schindler's List (1993)", 'The Lord of the Rings: The Return of the King (2003)']


In [18]:
recs_100_for_70000 = popular_recommendations('70000', 35, ranked_movies)
print(recs_100_for_70000)

['Selam (2013)', 'Filly Brown (2012)', 'CM101MMXI Fundamentals (2013)', 'Lawrence of Arabia (1962)', 'Airplane! (1980)', 'Yi dai zong shi (2013)', 'True Grit (1969)', 'The Princess Bride (1987)', 'Home Alone (1990)', 'The Shawshank Redemption (1994)', 'The Godfather (1972)', 'Metropolis (1927)', 'A Bridge Too Far (1977)', 'Il buono, il brutto, il cattivo. (1966)', 'My Name Is Khan (2010)', 'Double Indemnity (1944)', 'Akira (1988)', 'Terminator 2: Judgment Day (1991)', "Schindler's List (1993)", 'The Lord of the Rings: The Return of the King (2003)', 'Once Upon a Time in America (1984)', 'Incendies (2010)', 'Elizabeth (1998)', 'The Nightmare Before Christmas (1993)', 'Diarios de motocicleta (2004)', 'The Lord of the Rings: The Two Towers (2002)', 'Vertigo (1958)', 'Saving Private Ryan (1998)', 'The Dark Knight (2008)', 'The Lion King (1994)', 'The Incredibles (2004)', 'Festen (1998)', 'Heat (1995)', 'Paperman (2012)', "One Flew Over the Cuckoo's Nest (1975)"]


## Making Recommendation more Personalized

Adding **2 arguments** to make the recommentdation more personalized to user. First filter is the year, a user may want to see more recent or classic movies. Second filter is the genre of the movie, some may be more intrested in drama, and some more in comedy. 

In [28]:
def popular_recs_filtered(user_id, n_top, ranked_movies, years=None, genres=None):
 
    # Filter movies based on year and genre
    if years is not None:
        ranked_movies = ranked_movies[ranked_movies['Year'].isin(years)]

    if genres is not None:
        num_genre_match = ranked_movies[genres].sum(axis=1)
        ranked_movies = ranked_movies.loc[num_genre_match > 0, :]
            
            
    # create top movies list 
    top_movies = list(ranked_movies['movie'][:n_top])

    return top_movies

In [43]:
# Top 20 movies recommended for id 1 with years=['2015', '2016', '2017', '2018'], genres=['History']
recs_20_for_1_filtered = popular_recs_filtered('1', 20, ranked_movies, years=['2013', '2016', '2012', '2018'], genres=['Romance'])

for i, movie in enumerate(recs_20_for_1_filtered):
    print('{}. {}'.format(i+1, movie))

1. Paperman (2012)
2. Barfi! (2012)
3. The Perks of Being a Wallflower (2012)
4. En kongelig affære (2012)
5. La migliore offerta (2013)
6. Aashiqui 2 (2013)
7. Silver Linings Playbook (2012)
8. Les Misérables (2012)
9. Pee Mak Phrakanong (2013)
10. Moonrise Kingdom (2012)
11. De rouille et d'os (2012)
12. The Great Gatsby (2013)
13. Amour (2012)
14. Now Is Good (2012)
15. Song for Marion (2012)
16. Step Up Revolution (2012)
17. Safety Not Guaranteed (2012)
18. The Giant Mechanical Man (2012)
19. Pitch Perfect (2012)
20. Celeste & Jesse Forever (2012)
